<a href="https://colab.research.google.com/github/EveryTimeIWill18/Cython_Repo/blob/master/Cython_Tutorial_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cython Tutorial Three: *Memory Allocation*
---
C is one of the most efficient programming languages as C allows for dynamic memory allocation and reallocation. We can use __Cython__ to take full advantage of memory allocation to speed up our __Python__ applications.

## Pointers Review
---
Before we venture into the world of memory allocaiton, we need to review __pointers__.  Pointers are a variable type that is used to store the memory address of another variable. 
In __C__, this looks like:

```
int x = 10;
int* intPtr = &x;
```

In __Cython__:

```
cdef int x = 10;
cdef int* intPtr = &x;
```
Where:
  > __*__: declares __intPtr__ a pointer to an int.
  > __&__: is the memory address of a variable. In this case, __intPtr__ stores the memory address of __x__.

Since __intPtr__ points to the same memory location as __x__, if we change the value of __intPtr__ to 11, __x__ will be set to ll.

To get the value stored in __intPtr__, we must __*dereference*__ it by reusing the __*__ symbol again to get the value stored in __intPtr__. This can be confusing since we originally used __*__ to declare a pointer to an integer, but __C__ and __C++__ use __*__ for both declaring a pointer and dereferencing a pointer. __Cython__ is a bit simpler as we only use __*__ to declare the pointer and use square braket notation to dereference the pointer.
In __C__:

```
cdef int x = 10;  // set x to 10
cdef int* intPtr = &x; // create a pointer to an int and set it to the memory location of x
printf("the value of x is: %d\n", *intPtr); // dereference intPter to get the value
```

In [0]:
# load in the Cython extension
%load_ext cython

In [14]:
%%cython

cdef int x = 10 # set x to 10
cdef int* intPtr = &x # create a pointer to an int and set it to the memory location of x
print(intPtr[0]) # dereference the int pointer
intPtr[0] = 11  #change the value of x to 11
print(x)

10
11


## Using the C library *stdlib*
---
We will now introduce four functions from C' __stdlib__ library that are used for dynamic memory allocation. They are:

### malloc: *allocates a block of memory*
```int* intMem = (int*)malloc(10*sizeof(int)); // C usage```

```cdef int* intMem = <int*>malloc(10*sizeof(int)) # Cython usage```

### realloc: *attempts to resize a previously allocated block of memory that was allocated with __malloc__ or __calloc__*

```
intMem = (int*)realloc(int, 20); // C usage 
```

```
# Cython usage
cdef int* tmp = NULL # create a temporary pointer to an int
tmp = <int*>realloc(intMem, 20*sizeof(int)) # copy values of intMem into tmp and double its size
intMem = tmp # set intMem to the resized tmp pointer to an int
free(tmp) # free the memory allocated for tmp as its no longer needed

```

### calloc: *allocates memory and sets the memory to 0, unlike that of __malloc__*

```int *intMemTwo = (int*)calloc(10, sizeof(int)); // C usage```

```cdef int* intMemTwo = <int*>calloc(10, sizeof(int)) # Cython usage```

### free: deallocate the previously allocated memory

```free(intMem);  // C usage```

```free(intMem) # Cython usage```

In [0]:
%%cython
from libc.stdlib cimport malloc, free, realloc, calloc

cdef int* intMem = <int*>malloc(10*sizeof(int))  # allocate enough space for 10 integers
cdef int* mem = NULL                             # create a new pointer to an int and set to NULL
mem = <int*>realloc(intMem, 20*sizeof(int))      # use the realloc function from C's stdlib library
intMem = mem                                     # set intMem = mem
free(mem)                                        # free the temporary mem pointer to an int

cdef int* intMemTwo = <int*>calloc(10, sizeof(int)) # using calloc instead of malloc

## Void Pointers
---
Void pointers are useful for creating a generic pointer that can point to any type of variable.
Below is a __Cython__ example of how to use void pointers.

In [40]:
%%cython
# Using void pointers

cdef void* p  # this is a general purpose pointer. It currently has no type
cdef int x = 123  # create an int
p = &x            # void poitner points to an int
cdef int* px = <int*>p  # create a new pointer to an int and typecast p to a pointer to an int
print(px[0]) # dereference the point to an int

cdef double y = 1.123 # create a new double
p = &y

cdef double* py = <double*>p
print(py[0])

a = ['a', 'b', 'c']
cdef void* aPtr = <void*>a
print(<long>aPtr) # points to the same memory address of a
print(id(a))      # get the memroy address of a
print(<object>aPtr) # we can set the void poitner to the Python object type


123
1.123
140701548888200
140701548888200
['a', 'b', 'c']


## Using cpython.mem memory allocation functions
---
Instead of using C's __malloc__, __realloc__ and __free__, we can use __Python's__ __C-API__ memory allocation functions.

In [0]:
%%cython
from cpython.mem cimport PyMem_Malloc, PyMem_Realloc, PyMem_Free
cdef class AllocateMemory:
    """A class for efficient memory allocation"""

    cdef:
        int BUFFER         # size buffer for auto reallocation
        double* data
        int iterator       # start at the first position of data
        int data_length    # length of the array


    def __cinit__(self, size_t number):
        # allocate some memory
        self.iterator = 0 # init iterator
        self.BUFFER = 10  # init BUFFER
        self.data_length = number
        self.data = <double *> PyMem_Malloc(number * sizeof(double))
        self.data_length = number # set the current length of data
        if not self.data:
            raise MemoryError()

    def resize(self, size_t new_number):
        # Allocate new_number * sizeof(double) bytes
        # preserving the current content and making a best-effort to
        # re-use the original data location

        mem = <double *> PyMem_Realloc(self.data, new_number * sizeof(double))
        self.data_length = new_number # resize the length

        if not mem:
            raise MemoryError()
        # Only overwrite the pointer if the memory was really reallocated.
        # On error (mem is NULL), the originally memory has not been freed.
        self.data = mem

    def __dealloc__(self):
        PyMem_Free(self.data) # no-op if self.data is NULL


    def insert(self, double value, re_size=False, int new_number=-1):
        # insert data into the array
        if self.iterator < self.data_length:
            self.data[self.iterator] = value # set the value
            self.iterator += 1  # increment iterator
        elif self.iterator >= self.data_length:
            if re_size:
                if new_number > 0:
                    self.resize(new_number)
                    self.data[self.iterator] = value
                    self.iterator += 1  # increment iterator
                else:
                    print("The value, {} will not be inserted".format(value))
            else:
                raise TypeError()

    def c_print(self):
        print("iterator currently pointing to {}".format(self.iterator))

    def get_values(self):
        cdef counter = 0
        while counter < self.iterator:
            print(self.data[counter])
            counter += 1

### Using cymem memory allocation functions
---
The module, __cymem__ provides `cymem.Pool` that wraps around the `calloc` function so garbage collection is automated; meaning, we don't have to worry about calling the `free` function when we want to deallocate memory.

In [44]:
!pip install cymem

In [0]:
%%cython
from cymem.cymem cimport Pool
cdef Pool memory = Pool() # creates a Pool for storing memory addresses and frees them during garbage collection
cdef int* allocMemOne = <int*>memory.alloc(10, sizeof(int)) # allocate enough spacememroy for 10 ints
cdef int* allocMemTwo = <int*>memory.alloc(20, sizeof(int)) # allocate enough memory for 20 ints